Calcule a média por nome e sobre nome do ator da seguintes variaveis:
rental_duration
rental_rate
length
replacement_cost

Calcule a soma de amount(preço total do aluguel) por nome, sobrenome e email do cliente(customer):

In [10]:
import psycopg2 as pg2
import pandas as pd

In [11]:
def connect_to_postgres(dbname):
    conn = None
    try:
        conn = pg2.connect(host = 'localhost', 
                           port = 5432, 
                           dbname = dbname, 
                           user = 'postgres', 
                           password = 'dedaz123')
    except exception as e:
        print(e)
        conn = None
    return conn

def get_result_query(conn, query):
    data = None
    colnames = None
    try:
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
        data = cur.fetchall()
        colnames = [desc.name for desc in cur.description]
    except exception as e:
        print(e)
    cur.close()
    conn.close()
    return data, colnames

def to_df(data, colnames):
    return pd.DataFrame(data, columns=colnames)

In [12]:
dbname = 'dvdrental'
query = '''SELECT * FROM film as x
            LEFT JOIN film_actor as y
                on x.film_id = y.film_id
            LEFT JOIN actor as z
                on y.actor_id = z.actor_id'''
conn = connect_to_postgres(dbname)
data, colnames = get_result_query(conn, query)
df1 = to_df(data, colnames)

In [13]:
df1

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext,actor_id,film_id,last_update,actor_id,first_name,last_name,last_update
0,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'academi':1 'battl':15 'canadian':20 'dinosaur...,1.0,1.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
1,23,Anaconda Confessions,A Lacklusture Display of a Dentist And a Denti...,2006,1,3,0.99,92,9.99,R,2013-05-26 14:50:58.951,"[Trailers, Deleted Scenes]",'anaconda':1 'australia':18 'confess':2 'denti...,1.0,23.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
2,25,Angels Life,A Thoughtful Display of a Woman And a Astronau...,2006,1,3,2.99,74,15.99,G,2013-05-26 14:50:58.951,[Trailers],'angel':1 'astronaut':11 'battl':14 'berlin':1...,1.0,25.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
3,106,Bulworth Commandments,A Amazing Display of a Mad Cow And a Pioneer w...,2006,1,4,2.99,61,14.99,G,2013-05-26 14:50:58.951,[Trailers],'amaz':4 'bulworth':1 'command':2 'cow':9 'dis...,1.0,106.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
4,140,Cheaper Clyde,A Emotional Character Study of a Pioneer And a...,2006,1,6,0.99,87,23.99,G,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Behind the Scenes]",'ancient':19 'charact':5 'cheaper':1 'clyde':2...,1.0,140.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,958,Wardrobe Phantom,A Action-Packed Display of a Mad Cow And a Ast...,2006,1,6,2.99,178,19.99,G,2013-05-26 14:50:58.951,"[Trailers, Commentaries]",'action':5 'action-pack':4 'ancient':21 'astro...,200.0,958.0,2006-02-15 10:05:03,200.0,Thora,Temple,2013-05-26 14:47:57.620
5461,993,Wrong Behavior,A Emotional Saga of a Crocodile And a Sumo Wre...,2006,1,6,2.99,178,10.99,PG-13,2013-05-26 14:50:58.951,"[Trailers, Behind the Scenes]",'behavior':2 'cow':18 'crocodil':8 'discov':15...,200.0,993.0,2006-02-15 10:05:03,200.0,Thora,Temple,2013-05-26 14:47:57.620
5462,803,Slacker Liaisons,A Fast-Paced Tale of a A Shark And a Student w...,2006,1,7,4.99,179,29.99,R,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Behind the Scenes]",'ancient':21 'china':22 'crocodil':19 'fast':5...,NaN,NaN,NaT,NaN,None,None,NaT
5463,323,Flight Lies,A Stunning Character Study of a Crocodile And ...,2006,1,7,4.99,179,22.99,R,2013-05-26 14:50:58.951,[Trailers],'charact':5 'crocodil':9 'flight':1 'lie':2 'm...,NaN,NaN,NaT,NaN,None,None,NaT


In [16]:
df1['rental_rate'] = df1['rental_rate'].astype(float)
df1['replacement_cost'] = df1['replacement_cost'].astype(float)

In [17]:
df1[['first_name', 'last_name', 'rental_duration', 'rental_rate', 'length', 'replacement_cost']].groupby(['first_name', 'last_name']).mean()

rental_duration  rental_rate      length  \
first_name last_name                                             
Adam       Grant             5.277778     2.767778  122.388889   
           Hopper            4.863636     2.808182  112.272727   
Al         Garland           5.115385     2.759231  106.384615   
Alan       Dreyfuss          5.037037     2.841852  111.296296   
Albert     Johansson         5.060606     3.293030  126.636364   
...                               ...          ...         ...   
Will       Wilson            5.064516     3.054516  115.967742   
William    Hackman           4.777778     3.138148  129.074074   
Woody      Hoffman           4.580645     2.990000  120.838710   
           Jolie             4.354839     2.602903  114.935484   
Zero       Cage              4.840000     2.830000  113.240000   

                      replacement_cost  
first_name last_name                    
Adam       Grant             22.934444  
           Hopper            19.490000  
Al         Garland           19.874615  
Alan       Dreyfuss          19.360370  
Albert     Johansson         22.474848  
...                                ...  
Will       Wilson            19.957742  
William    Hackman           20.064074  
Woody      Hoffman           19.699677  
           Jolie             19.409355  
Zero       Cage              19.910000  

[199 rows x 4 columns]

In [18]:
query = '''SELECT * FROM customer as x
            LEFT JOIN payment as y
                on x.customer_id = y.customer_id'''
conn = connect_to_postgres(dbname)
data, colnames = get_result_query(conn, query)
df2 = to_df(data, colnames)

In [19]:
df2

,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577
1,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577
2,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577
3,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577
4,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14591,245,1,Courtney,Day,courtney.day@sakilacustomer.org,249,True,2006-02-14,2013-05-26 14:49:45.738,1,32094,245,2,12682,2.99,2007-05-14 13:44:29.996577
14592,251,2,Vickie,Brewer,vickie.brewer@sakilacustomer.org,255,True,2006-02-14,2013-05-26 14:49:45.738,1,32095,251,1,14107,0.99,2007-05-14 13:44:29.996577
14593,252,2,Mattie,Hoffman,mattie.hoffman@sakilacustomer.org,256,True,2006-02-14,2013-05-26 14:49:45.738,1,32096,252,2,13756,4.99,2007-05-14 13:44:29.996577
14594,263,1,Hilda,Hopkins,hilda.hopkins@sakilacustomer.org,268,True,2006-02-14,2013-05-26 14:49:45.738,1,32097,263,1,15293,0.99,2007-05-14 13:44:29.996577


In [21]:
df2[['first_name', 'last_name', 'amount', 'email']].groupby(['first_name', 'last_name', 'email']).sum().sort_values('amount')

,,,amount
first_name,last_name,email,
Brian,Wyman,brian.wyman@sakilacustomer.org,27.93
Leona,Obrien,leona.obrien@sakilacustomer.org,32.90
Caroline,Bowman,caroline.bowman@sakilacustomer.org,37.87
Anthony,Schwab,anthony.schwab@sakilacustomer.org,47.85
Tiffany,Jordan,tiffany.jordan@sakilacustomer.org,49.88
...,...,...,...
Clara,Shaw,clara.shaw@sakilacustomer.org,189.60
Rhonda,Kennedy,rhonda.kennedy@sakilacustomer.org,191.62
Marion,Snyder,marion.snyder@sakilacustomer.org,194.61
